In [43]:
import os
import requests
import pandas as pd

from bs4 import BeautifulSoup
from openpyxl import load_workbook
from markdownify import markdownify
from dotenv import load_dotenv

load_dotenv()


True

In [ ]:
CONTENT_URL = os.getenv("CLIC_SHAREPOINT_URL")
FILE_SAVE_PATH = "data/CLIC-Content.xlsx"
LANGUAGE_CODE = "EN"
# LANGUAGE_CODE = "ZH"

COLS = {
    "EN": ["nid", "Result", "Completed", "title", "full_path", "topic", "content"],
    "ZH": ["nid", "Result", "Completed", "title", "path", "topic", "new content"],
}
COL_NAMES = ["nid", "type", "is_complete", "title", "path", "topic_value", "content"]

assert CONTENT_URL, "SHAREPOINT_URL must be set in .env file"
os.makedirs(os.path.dirname(FILE_SAVE_PATH), exist_ok=True)

## Read the data

#### Download the content from SharePoint

In [ ]:
url = CONTENT_URL + "?download=1"

response = requests.get(url, stream=True)
total_size_in_bytes = int(response.headers.get("content-length", 0))
block_size = 1024
with open(FILE_SAVE_PATH, 'wb') as f:
    counter = 0
    for data in response.iter_content(block_size):
        f.write(data)
        counter += 1

#### Check for removed rows
- Indicated by strikethrough formatting

In [46]:
ws = load_workbook(FILE_SAVE_PATH)[LANGUAGE_CODE]
rows_to_remove = set()
# Check each cell for strikethrough formatting
for row_idx, row in enumerate(ws.iter_rows(min_row=2)):  # Skip header row
    for cell in row:
        if cell.font and cell.font.strike:
            rows_to_remove.add(row_idx)
            break
print(f"Rows to remove: {rows_to_remove}")

Rows to remove: {130, 1672, 909, 1040, 2449, 914, 915, 1567, 1185, 2298, 171, 2607, 48, 2608, 2609, 2610, 2611, 2612, 2613, 2614, 1080, 2615, 2616, 2617, 2618, 2619, 838, 455, 458, 603, 1376, 1377, 227, 228, 2275, 614, 870, 872, 745, 1388, 1389, 878, 1390, 1392, 1268, 1397, 1146, 125, 126}


#### Load Dataframe and clean

In [47]:
df = pd.read_excel(FILE_SAVE_PATH, sheet_name=LANGUAGE_CODE, usecols=COLS[LANGUAGE_CODE])
df.columns = COL_NAMES
df.drop(index=rows_to_remove, inplace=True)
df.dropna(subset=["title", "content", "topic_value"], inplace=True) 
df.drop(index=df[~df["path"].str.contains("topics/")].index, inplace=True)              # filter out announcement pages

df['url'] = df['path']
df['path'] = df['path'].apply(lambda x: '/topics/' + x.split('/topics/')[1] if '/topics/' in x else x)

# df.drop(index=df[df["type"] == "Index"].index, inplace=True)                              # filter out index pages
df["is_noisy"] = df["type"] == "Index"  # Mark noisy pages

df.drop(index=df[df["type"] == "Delete"].index, inplace=True)                               # filter out deleted pages
df['nid'] = df['nid'].astype(int)                                                     # Convert 'nid' to int
df['is_complete'] = df['is_complete'].fillna(0).astype(bool)                    # Convert 'is_complete' to boolean (1.0 -> True, NaN/0.0 -> False)
df.head(3)

,nid,type,is_complete,title,path,topic_value,content,url,is_noisy
0,8,NaN,True,Criminal liability and types of penalties,/topics/PoliceAndCriminalProcedure/criminal_li...,Police & Criminal Procedure,<h2>I. Criminal liability and types of penalti...,https://clic.org.hk/en/topics/PoliceAndCrimina...,False
1,9,Index,False,Criminal records and the Rehabilitation of Off...,/topics/PoliceAndCriminalProcedure/Criminal-Re...,Police & Criminal Procedure,<h2>II. Criminal records and the Rehabilitatio...,https://clic.org.hk/en/topics/PoliceAndCrimina...,True
2,10,NaN,True,Police powers,/topics/PoliceAndCriminalProcedure/police_powers,Police & Criminal Procedure,<h2>III. Police powers</h2> <p>As an ordinary ...,https://clic.org.hk/en/topics/PoliceAndCrimina...,False


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2541 entries, 0 to 2637
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nid          2541 non-null   int64 
 1   type         312 non-null    object
 2   is_complete  2541 non-null   bool  
 3   title        2541 non-null   object
 4   path         2541 non-null   object
 5   topic_value  2541 non-null   object
 6   content      2541 non-null   object
 7   url          2541 non-null   object
 8   is_noisy     2541 non-null   bool  
dtypes: bool(2), int64(1), object(6)
memory usage: 163.8+ KB


## Extract CLIC pages' topics keys

In [49]:
def getTopicFromPath(path):
    if isinstance(path, str) and 'topics/' in path:
        x = path.split('topics/')[1]
        return x.split('/')[0] if '/' in x else x
    return None

df['topic_key'] = df['path'].apply(getTopicFromPath)
df.head(3)

,nid,type,is_complete,title,path,topic_value,content,url,is_noisy,topic_key
0,8,NaN,True,Criminal liability and types of penalties,/topics/PoliceAndCriminalProcedure/criminal_li...,Police & Criminal Procedure,<h2>I. Criminal liability and types of penalti...,https://clic.org.hk/en/topics/PoliceAndCrimina...,False,PoliceAndCriminalProcedure
1,9,Index,False,Criminal records and the Rehabilitation of Off...,/topics/PoliceAndCriminalProcedure/Criminal-Re...,Police & Criminal Procedure,<h2>II. Criminal records and the Rehabilitatio...,https://clic.org.hk/en/topics/PoliceAndCrimina...,True,PoliceAndCriminalProcedure
2,10,NaN,True,Police powers,/topics/PoliceAndCriminalProcedure/police_powers,Police & Criminal Procedure,<h2>III. Police powers</h2> <p>As an ordinary ...,https://clic.org.hk/en/topics/PoliceAndCrimina...,False,PoliceAndCriminalProcedure


In [50]:
topics = df['topic_key'].unique()
print(f"Number of topics in data: {len(topics)}")
topics

Number of topics in data: 37


array(['PoliceAndCriminalProcedure', 'hkLegalSystem', 'ADR', 'civilCase',
       'legalAid', 'personalInjuries',
       'bankruptcy_IndividualVoluntaryArrangement_Companies_Winding_up',
       'enduring_Powers_of_Attorney', 'DIY_Residential_Tenancy_Agreement',
       'Maintenance_and_safety_of_property', 'landlord_tenant',
       'Redevelopment_and_Acquisition_of_Property',
       'saleAndPurchaseOfProperty', 'probate', 'taxation', 'insurance',
       'Medical_treatment_consent_and_withdrawal', 'medicalNegligence',
       'sexual_offences', 'traffic_law', 'immigration', 'Competition_Law',
       'businessAndCommerce', 'employmentDisputes',
       'intellectualProperty', 'FreedomOfAssemblyProcessionDemonstration',
       'familyMatrimonialAndCohabitation', 'personalDataPrivacy',
       'protectionForInvestorsAndStructuredProducts',
       'consumer_complaints', 'antiDiscrimination', 'defamation',
       'offences_related_to_dangerous_drugs', 'judicial_review',
       'consumerCredit', '

## Title modification

#### Remove list index in title

In [51]:
separators = [')', '.', '. ']

def cleanTitle(title: str) -> str:    
    separator = None
    for sep in separators:
        if sep in title[:5]:
            separator = sep
            break
        
    if separator is None:
        return title.strip()
    
    return title.split(separator, 1)[-1].strip()

df['title'] = df['title'].apply(cleanTitle)

#### Check if title is question

In [52]:
df['is_question'] = df['title'].apply(lambda x: x.endswith('?'))

is_question_count = len(df[df['is_question'] == True])
print(f"Number of questions: {is_question_count}, out of {len(df)} total rows.")

Number of questions: 1414, out of 2541 total rows.


In [53]:
import re
def parseCamel(text):
    if len(text) < 5:
        return text
    # Split on uppercase letters, but not at the start of the string
    x = re.split(r'(?<!^)(?=[A-Z])', text)
    if len(x) > 1:
        if x[0].isnumeric():
            x = x[1:]  # If the first part is numeric, skip it
        else:
            x[0] = x[0][0].upper() + x[0][1:]
    return " ".join(x)

def parseSnake(text):
    # x = re.sub('(_)([a-zA-Z0-9])', lambda x: ' ' + x.group(2).upper(), text)
    x = text.split('_')
    if len(x) > 1 and x[0].isnumeric():
        x = x[1:]  # If the first part is numeric, skip it
    return " ".join(x)


# Example path: https://clic.org.hk/zh/topics/FreedomOfAssemblyProcessionDemonstration/RelatedOffences/Public_Order_Offences/Disorder_in_public_places
def getContext(path: str, topic: str) -> str:
    ctx = path.split('topics/')[1].split('/')[1:-1]
    ctx = [topic] + [parseSnake(x) if '_' in x else parseCamel(x) for x in ctx]
    return ' > '.join(ctx)

# getContext('https://clic.org.hk/en/topics/ADR/Mediation/relevantPracticeDirections/pd18.1&18.2', 'Freedom Of Assembly Procession Demonstration')
df['context'] = df.apply(lambda x: getContext(x['path'], x['topic_value']), axis=1)

## Extract Content

In [54]:
import re
def normalize_newlines(text):
    # Replace 2 or more consecutive newlines with exactly two newlines
    return re.sub(r'(\s*\n\s*){2,}', '\n\n', text)

#### Remove header line and convert to markdown

In [55]:
content_no_header = df['content'].apply(lambda x: x.split("</h2>", maxsplit=1)[1].strip() if len(x.split("</h2>")) > 1 else x)
df['parsed_content'] = content_no_header.apply(lambda x: normalize_newlines(markdownify(x).strip()))
print(df.head(1))
del content_no_header

   nid type  is_complete                                      title  \
0    8  NaN         True  Criminal liability and types of penalties   

                                                path  \
0  /topics/PoliceAndCriminalProcedure/criminal_li...   

                   topic_value  \
0  Police & Criminal Procedure   

                                             content  \
0  <h2>I. Criminal liability and types of penalti...   

                                                 url  is_noisy  \
0  https://clic.org.hk/en/topics/PoliceAndCrimina...     False   

                    topic_key  is_question                      context  \
0  PoliceAndCriminalProcedure        False  Police & Criminal Procedure   

                                      parsed_content  
0  Criminal liability is generally made up of two...  


#### Extract CLIC, legislation and case references

In [56]:
BASE_URL = f"https://clic.org.hk/{LANGUAGE_CODE.lower()}/topics/"
def find_new_path(path: str, retries=0) -> str:
    '''
    For some pages the path has changed, but some hyperlinks still point to the old path.
    This function finds the new URL based on the old path.
    '''
    if not isinstance(path, str):
        return None
    url = BASE_URL + path
    try:
        response = requests.get(url, timeout=5, allow_redirects=True)
        if response.status_code in [200, 301]:
            # print(f"Found new path for {path}: {response.url}")
            return response.url.split('/topics/')[1].strip() if '/topics/' in response.url else None
        else:
            raise requests.RequestException(f"Failed to fetch {url}: {response.status_code}")
    except requests.RequestException:
        if retries < 3:
            return find_new_path(path, retries + 1)
    return None

In [57]:
df['cases_ref'] = None
df['legislation_ref'] = None
df['clic_ref'] = None

for i, col in df.iterrows():
    soup = BeautifulSoup(col["content"], 'html.parser')
    cap = []
    cases = []
    clic = []

    links = soup.find_all('a')
    for link in links:

        if link.has_attr('href') and "hklii" in link['href']:
            # hklii links can be cases or legislation
            url = link['href'].split('?')[0] if "?" in link['href'] else link['href']
            url = url[:-5] if link['href'].endswith('.html') else url
            
            if 'cases' in link['href']:
                # Parse Cases (https://www.hklii.hk/en/cases/hkca/2020/124 or https://www.hklii.org/cgi-bin/sinodisp/eng/hk/cases/hkcfi/2018/2243.html)
                metadata = url.split('/cases')[1].strip().split('/')
                if len(metadata) >= 3:
                    cases.append({
                        'court': metadata[1],
                        'year': metadata[2],
                        'no': metadata[3]
                    })
            else:
                # Parse Legislation (http://www.hklii.hk/hk/legis/en/ord/344/ or https://www.hklii.org/eng/hk/legis/reg/382A/s12.html or http://www.hklii.hk/eng/hk/legis/instrument/101/)
                leg_type = 'ord' if '/ord/' in url else 'reg' if '/reg/' in url else 'instrument' if '/instrument/' in url else None
                if leg_type is None: continue

                metadata = url.split(f'/{leg_type}/')[1].strip().split('/')
                if len(metadata) >= 1:
                    cap.append({
                        'type': leg_type,
                        'no': metadata[0] if leg_type != 'instrument' else f"A{metadata[0]}" if metadata[0].isdigit() else metadata[0],
                        'section': metadata[1] if len(metadata) > 1 else ""
                    })
        
        elif link.has_attr('href') and "/topics/" in link['href']:
            # References to other pages on the CLIC site
            page_path = link['href']
            if "#" in page_path:
                # Remove fragment identifiers (e.g., #section1)
                page_path = page_path.split('#')[0]
            page_path = page_path.split('/topics/')[1].strip()
            if page_path.endswith('/'):
                page_path = page_path[:-1]

            # Find the corresponding nid in the DataFrame
            page_nids = df[df['path'].str.endswith(page_path)]['nid'].values
            if len(page_nids) == 0:
                # If no nid found, try to find the new URL
                new_url = find_new_path(page_path)
                if new_url:
                    page_nids = df[df['path'].str.endswith(new_url)]['nid'].values

            clic.append({
                'nid': page_nids[0] if len(page_nids) > 0 else None,
                'path': '/topics/' + page_path.strip(),
            })
            
    df.at[i, 'cases_ref'] = cases
    df.at[i, 'legislation_ref'] = cap
    df.at[i, 'clic_ref'] = clic

#### Count number of tokens in each row

In [58]:
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
df['n_tokens'] = df['parsed_content'].apply(lambda x: len(enc.encode(x)))

In [59]:
over_1024_texts = df[df['n_tokens'] > 1024]
print(f"Number of long texts (more than 1024 tokens): {len(over_1024_texts)}")

over_512_texts = df[df['n_tokens'] > 512]
print(f"Number of long texts (more than 512 tokens): {len(over_512_texts)}")

over_256_texts = df[df['n_tokens'] > 256]
print(f"Number of long texts (more than 256 tokens): {len(over_256_texts)}")

under_12_texts = df[df['n_tokens'] < 12]
print(f"Number of short texts (less than 12 tokens): {len(under_12_texts)}")

Number of long texts (more than 1024 tokens): 141
Number of long texts (more than 512 tokens): 504
Number of long texts (more than 256 tokens): 1125
Number of short texts (less than 12 tokens): 208


In [60]:
df['is_noisy'] = df['n_tokens'] < 12

In [61]:
df[df['n_tokens'] > 512].to_excel("data/CLIC_content_512_tokens.xlsx", index=False)

In [62]:
# df.to_excel("data/CLIC_content_cleaned.xlsx", index=False)
df.to_json("data/CLIC_content_cleaned.json", orient="records", indent=2, force_ascii=False)
